<h3 style="text-align:center;color:cadetblue;">Working with databases</h3>

1. Working With Databases
    - SQLite basics
    - Using `with` to Manage Your Database Connection
    - Working With Database Tables
    - Executing Multiple SQL Statements
    - Avoid Security Issues With Parametrized
    - Retrieving Data
    - Libraries for Working With Other SQL Databases
    - Review Exercises

<p style="text-align:center;color:blue;"><b>Working With Databases</b></p>

A database is a structured system for storing data. It could be made up of several CSV files organized into directories or something more elaborate. A database management system is software that manages access to and interacts with a database.

Python comes with a lightweight database management system called `SQLite` that is perfect for learning how to work with databases.

SQLite uses **structured query language** (SQL) to interact with its database. <u>Some experience with SQL will be helpful when reading this lesson.</u>

### An Introduction to SQLite
There are numerous SQL database engines, and some are better suited to particular purposes than others. One of the simplest, most lightweight SQL database engines is SQLite, which runs directly on your machine and comes bundled with the standard Python installation.

### SQLite Basics

There are four basic steps to working with SQLite:
1. Import the sqlite3 package.
2. Connect to an existing database or create a new database.
3. Execute SQL statements on the database.
4. Close the database connection.

In [1]:
import sqlite3

connection = sqlite3.connect("test_database.db")

The `sqlite3.connect()` function is used to connect to or create a database.

When you execute `.connect("test_database.db")`, Python searches for an existing database called "test_database.db". If no database with that name is found, a new one is created in the current working directory.

To create a database in a different directory, you must specify the full path in the argument to `.connect()`.

You can create an in-memory database by passing the string `":memory:"` to `.connect()`: 

    `connection = sqlite3.connect(":memory:")`

This is a good way to store data that only needs to exist while a program is running.

`.connect()` returns a `sqlite3.Connection` object. You can verify this with type():

In [ ]:
type(connection)

The Connection object represents the connection between your program and the database. It has several attributes and methods that you can use to interact with the database.

To store and retrieve data, you need a Cursor object, which you can
obtain with `connection.cursor()`:

In [ ]:
cursor = connection.cursor()
type(cursor)

The `sqlite3.Cursor` object is your gateway to interacting with the database. Using a Cursor, you can create database tables, execute SQL statements, and fetch query results.

In database jargon, the term cursor refers to an object that is used to fetch results from a database query one row at a time.

Let’s use the SQLite datetime() function to get the current local time:

In [ ]:
query = "SELECT datetime('now', 'localtime')"
results = cursor.execute(query)
results

"SELECT datetime('now', 'localtime');" is an SQL statement that returns the current date and time. You assign the query to the `query` variable and pass it to `cursor.execute()`. This runs the query against the database and returns a `Cursor` object, which you assign to the results
variable.

You might be wondering where the time returned by `datetime()` is. To get the query results, use `results.fetchone()`, which returns a tuple containing the first row of results:


In [ ]:
row = results.fetchone()
row

Since `.fetchone()` returns a tuple, you need to access the first element to get the string containing the date and time information:

In [ ]:
time = row[0]
time

Finally, call `connection.close()` to close the database connection:


In [7]:
connection.close()

> Note: It’s important to always close database connections when you’re done using them to avoid leaving system resources hanging around after your program stops running.

### Using `with` to Manage Your Database Connection

You can use a `with` statement with `open()` to open a file and then automatically close it once the with block has executed. The same pattern applies to SQLite database connections and is the recommended way to open a database connection.

Here’s the `datetime()` example from above using a with statement to
manage the database connection:


In [ ]:
with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    query = "SELECT datetime('now', 'localtime');"
    results = cursor.execute(query)
    row = results.fetchone()
    time = row[0]

time

In this example, you assign the Connection object returned by `sqlite3.connect()` to the `connection` variable in the with statement. The code in the with block creates a new `Cursor` object using `connection.cursor()` and then gets the current time with the `Cursor` object’s .`execute()` and `.fetchone()` methods.

Managing your database connections in a `with` statement has many advantages. The resulting code is often cleaner and shorter than code written without a `with` statement.

### Working With Database Tables

You don’t usually want to create a whole database just to get the current time. Databases are used to store and retrieve information. To store data in a database, you need to create a table and write some values to it.

Let’s create a table called People with three columns: FirstName, LastName, and Age. The SQL query to create this table looks like this:

```
CREATE TABLE People(FirstName TEXT, LastName TEXT, Age INT);
```

Notice that FirstName and LastName are followed by the word TEXT, whereas Age is followed by the word INT. This tells SQLite that values in the FirstName and LastName columns are text values, while values in the Age column are integers.

Once you create the table, you can populate it with data using the INSERT INTO SQL command. The following query inserts the values Ron, Obvious, and 42 in the FirstName, LastName, and Age columns, respectively:

```
INSERT INTO People VALUES('Ron', 'Obvious', 42);

```

Note that the strings 'Ron' and 'Obvious' are delimited with single quotation marks. This still makes them valid Python strings, but more importantly, only strings delimited with single quotes are valid SQLite strings.

When you write SQL queries as Python strings, you need to make sure that they’re delimited with double quotation marks so that you can use single quotation marks inside them to delimit SQLite strings.

SQLite is not the only SQL database management system that follows the single quote convention. Keep an eye out for this whenever you work with any SQL database.

Let’s walk through how to execute these statements and save the changes to the database. First, we’ll do it without using a with statement.

In [9]:
import sqlite3
create_table = """
    CREATE TABLE People(
    FirstName TEXT,
    LastName TEXT,
    Age INT
    );"""
insert_values = """
    INSERT INTO People VALUES(
    'Ron',
    'Obvious',
    42
    );"""
connection = sqlite3.connect("test_database.db")
cursor = connection.cursor()
cursor.execute(create_table)
cursor.execute(insert_values)
connection.commit()
connection.close()

First, you create two strings containing SQL statements for creating the People table and inserting some data into it. You assign these strings to the create_table and insert_values variables.

You write both SQL statements using triple-quoted strings so you can format the SQL nicely. SQL ignores whitespace, so you can use spacing in the string to improve the readability of your Python code.

Then you create a Connection object with sqlite3.connect() and assign it to the connection variable. You also create a Cursor object with connection.cursor() and use it to execute the two SQL statements.

In [ ]:
connection = sqlite3.connect("test_database.db")
cursor = connection.cursor()
query = "SELECT * FROM People;"
results = cursor.execute(query)
results.fetchone()

Now let’s rewrite the program using a with statement to manage the database connection.

Before you can do anything, though, you need to delete the People table so that you can re-create it. Type the following code into the interactive window to remove the People table from the database:

In [ ]:
cursor.execute("DROP TABLE People;")

In [12]:
connection.commit()
connection.close()

Change the program as follows:

In [13]:
import sqlite3

create_table = """
    CREATE TABLE People(
    FirstName TEXT,
    LastName TEXT,
    Age INT
    );"""

insert_values = """
    INSERT INTO People VALUES(
    'Ron',
    'Obvious',
    42
    );"""

with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    cursor.execute(create_table)
    cursor.execute(insert_values)

Neither connection.close() nor connection.commit() is needed. Any changes you make to the database will be automatically committed when the with block is done executing. This is another advantage of using a with statement to manage your database connection.

### Executing Multiple SQL Statements

An SQL script is a collection of SQL statements separated by semicolons (;) that can be run all at the same time. Cursor objects have an .executescript() method for executing SQL scripts.

The following program executes an SQL script that creates a People table and inserts some values into it:

In [14]:
import sqlite3

sql = """
DROP TABLE IF EXISTS People;
CREATE TABLE People(
    FirstName TEXT,
    LastName TEXT,
    Age INT
    );
INSERT INTO People VALUES(
    'Ron',
    'Obvious',
    '42'
    );"""
with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    cursor.executescript(sql)

You can also execute many similar statements by using the `.executemany()` method and supplying a tuple of tuples in which each inner tuple supplies the information for a single command.

For instance, if you have a lot of people’s information to insert into your People table, you can save this information in the following tuple of tuples:


In [15]:
people_values = (
("Ron", "Obvious", 42),
("Luigi", "Vercotti", 43),
("Arthur", "Belling", 28)
)

In [ ]:
cursor.executemany("INSERT INTO People VALUES(?, ?, ?)", people_values)

In [17]:
connection.commit()

You can then insert all these people at once in a single line of code:

```
cursor.executemany("INSERT INTO People VALUES(?, ?, ?)", people_values)
```

Here, the question marks act as placeholders for the tuples contained in people_values. This is called a parameterized statement.

Each ? represents a parameter that gets replaced by a value from people_values when the method is executed. The parameters are replaced in order. That is, the first ? is replaced by the first value in people_values, the second ? is replaced by the second value, and so on.


### Avoid Security Issues With Parametrized 

For security reasons, especially when you need to interact with an SQL table based on user input, you should always use parameterized SQL statements. This is because the user could potentially supply a value that looks like SQL code and causes your SQL statement to behave in unexpected ways. This is called an **SQL injection attack** and, even if you aren’t dealing with a malicious user, it can happen entirely by accident.

For example, suppose you want to insert a person into the People table based on user-supplied information. You might initially try something like the following:

In [18]:
import sqlite3
# Get person data from user
first_name = input("Enter your first name: ")
last_name = input("Enter your last name: ")
age = int(input("Enter your age: "))
# Execute insert statement for supplied person data
query = (
    "INSERT INTO People Values"
    f"('{first_name}', '{last_name}', {age});"
)
with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    cursor.execute(query)


What if the user’s name includes an apostrophe? Try adding Flannery O’Connor to the table, and you’ll see that she breaks the code. This is because the apostrophe gets mixed up with the single quotes in the line, making it appear that the SQL code ends earlier than you intend.

In this case, the code only causes an error, which is bad enough. In some cases, though, bad input can corrupt an entire table. Many other hard-to-predict cases can break SQL tables and even delete portions of your database. To avoid this, you should always use parameterized
statements.

The following code uses a parametrized statement to safely insert the user input into the database:


In [ ]:
import sqlite3
first_name = input("Enter your first name: ")
last_name = input("Enter your last name: ")
age = int(input("Enter your age: "))
data = (first_name, last_name, age)
with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    cursor.execute("INSERT INTO People VALUES(?, ?, ?);", data)

Parametrization is also useful for updating a row in the database with an SQL UPDATE statement:

```
cursor.execute(
    "UPDATE People SET Age=? WHERE FirstName=? AND LastName=?;",
    (45, 'Luigi', 'Vercotti')
    )

```

This code updates the value of the Age column to 45 for the row in which FirstName is set to 'Luigi' and LastName is set to 'Vercotti'.

### Retrieving Data

Inserting and updating information in a database isn’t all that helpful if you can’t fetch that information from the database.

To fetch data from a database, you can use the `.fetchone()` and `.fetchall()` cursor methods. The `.fetchone()` method returns a single row from query results, while `.fetchall()` retrieves all the results of a query at once.

The following program illustrates how to use `.fetchall()`:

In [ ]:
import sqlite3
values = (
    ("Ron", "Obvious", 42),
    ("Luigi", "Vercotti", 43),
    ("Arthur", "Belling", 28),
)
with sqlite3.connect("test_database.db") as connection:
    cursor = connection.cursor()
    cursor.execute("DROP TABLE IF EXISTS People")
    cursor.execute("""
        CREATE TABLE People(
        FirstName TEXT,
        LastName TEXT,
        Age INT
        );"""
    )
cursor.executemany("INSERT INTO People VALUES(?, ?, ?);", values)
# Select all first and last names from people over age 30
cursor.execute(
"SELECT FirstName, LastName FROM People WHERE Age > 30;"
)
for row in cursor.fetchall():
    print(row)


In the above program, you first drop the People table to destroy the changes made in the previous examples in this section. Then you re-create the People table and insert several values into it. Next, you `.execute()` a SELECT statement that returns the first and last names of all people over age 30.

Finally, `.fetchall()` returns the results of your query as a list of tuples in which each tuple contains a single row from the query results.

If you type the program into a new editor window and save and run the file, then you’ll see the following output displayed in the interactive window:
```
('Ron', 'Obvious')
('Luigi', 'Vercotti')
```
Indeed, Ron and Luigi are the only people in the database who are over thirty years old.

### Libraries for Working With Other SQL Databases

If you have a particular type of SQL database that you’d like to access through Python, most of the basic syntax is likely to be identical to what you just learned for SQLite. However, you’ll need to install an additional package to interact with your database since SQLite is the
only built-in option.

Many SQL variants and corresponding Python packages are available. Here are a few of the most commonly used and reliable open source alternatives to SQLite:
-  `pyodbc` connects to ODBC (Open Database Connectivity) databases, such as Microsoft SQL Server.
-  `psycopg2` connects to PostgreSQL databases.
- `PyMySQL` connects to MySQL databases.

One difference between SQLite and other database engines—besides the actual syntax of the SQL code, which changes slightly with most flavors of SQL—is that most database engines require a username and a password to connect. Check the documentation for the particular package you want to use to get the proper syntax for making a database connection.

The `SQLAlchemy` package is another popular option for working with databases. SQLAlchemy is an object-relational mapper, or ORM, that uses an object-oriented paradigm to build database queries. It can be configured to connect to a variety of databases. The object-oriented approach allows you to make queries without writing raw SQL statements.